In [1]:
from glob import glob
from framed import load_cbmodel
import pandas as pd
import numpy as np
from framed.io.sbml import sanitize_id

import reframed
from reframed.cobra.ensemble import load_ensemble

import cobra
from cobra.io import read_sbml_model
import pandas as pd

In [2]:
NeuroMetabDF=pd.read_excel('NeuroMetab.xlsx')
NeuroMetab=NeuroMetabDF['BiGGIDnoM'].tolist()
print('list of neuroactive metabolites:\n', NeuroMetab)
noNeuroMetab=len(NeuroMetab)
print('number of neuroactive metabolites in the universal model:', noNeuroMetab)

list of neuroactive metabolites:
 ['trp__L_p', 'trp__L_e', 'trp__L_c', 'trypta_c', '34dhphe_p', '34dhphe_e', '34dhphe_c', 'tym_p', 'tym_e', 'gg4abut_c', 'dopa_p', 'dopa_e', 'dopa_c', 'glu__L_c', 'glu__L_e', 'glu__L_p', 'no_p', 'no_e', 'no_c', 'taur_p', 'taur_e', 'taur_c', 'but_c', 'but_e', 'but_p', 'ppa_c', 'ppa_e', 'ppa_p', 'ac_c', 'ac_e', 'ac_p', '3mb_c', '3mb_p', '3mb_e', '3mba_c', '3mba_e', 'inost_c', 'inost_e', 'inost_p', 'mqn6_c', 'mqn7_c', 'mqn8_c', 'quln_c', 'amet_c']
number of neuroactive metabolites in the universal model: 44


In [3]:
UniversalModel=read_sbml_model("universe_bacteria.xml")

ModelMetabs=[]
ReactionsIdData=[]
ReactionsIdDataIndexNames=[]
TransportersINneuroRXs=[]
TransptGPRList=[]
UMetabList=[]

for x in UniversalModel.metabolites: #for each metabolite of the model metabolites
    UMetabList.append(x.id)

for NeuroactiveMetab in NeuroMetab:
    NeuroactiveMetab=NeuroactiveMetab.replace(" ' ", '')
    
    reactionsID=[]
    genesList=[]

    producingDF=UniversalModel.metabolites.get_by_id(NeuroactiveMetab).summary().producing_flux
    consumingDF=UniversalModel.metabolites.get_by_id(NeuroactiveMetab).summary().consuming_flux

    if producingDF.empty==False:
        slicedProducingDF=producingDF[['reaction']]
        listOfProdStrings = ['producing'] * slicedProducingDF.shape[0]
        slicedProducingDF.insert(1, "RxType", listOfProdStrings, True)

    if consumingDF.empty==False:
        slicedConsumingDF=consumingDF[['reaction']]
        listOfConsumStrings = ['consuming'] * slicedConsumingDF.shape[0]
        slicedConsumingDF.insert(1, "RxType", listOfConsumStrings, True)

        AllRXsDF = pd.concat([slicedProducingDF, slicedConsumingDF])
# print('dataframe with all the reactions \n:', AllRXsDF)

        for index, rows in AllRXsDF.iterrows():    # Put REACTIONids in a list so that we can later loop over it
            RowIndex = str(NeuroactiveMetab)
            Row =[rows.reaction, RowIndex, rows.RxType]
            ReactionsIdData.append(Row)

ReactionsIdDf=pd.DataFrame(ReactionsIdData, columns=['reaction',  'metabolite', 'RxType'])
ReactionsIdDf=ReactionsIdDf.sort_values('reaction')
print('dataframe with reactions \n:', ReactionsIdDf)

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
dataframe with reactions 
:        reaction metabolite     RxType
27   34DHPHEtex  34dhphe_e  producing
25   34DHPHEtex  34dhphe_p  producing
28   34DHPHEtex  34dhphe_e  consuming
30   34DHPHEtpp  34dhphe_c  producing
26   34DHPHEtpp  34dhphe_p  consuming
..          ...        ...        ...
131       VALTA   glu__L_c  producing
15      araphe3   trp__L_c  producing
22      aratry1   trp__L_c  consuming
16      aratry2   trp__L_c  producing
17      aratyr4   trp__L_c  producing

[445 rows x 3 columns]


In [5]:
NeuroUniversalRxs=ReactionsIdDf.reaction.unique()


# # print(NeuroUniversalRxs)

NeuroUniversalRxsDF = pd.DataFrame(NeuroUniversalRxs)
# NeuroUniversalRxsDF
# # print(NeuroUniversalRxsDF)

# # NeuroUniversalRxsDF.to_excel("UniversalNeuroactiveRxs.xlsx")

# NeuroUniversalRxs=NeuroUniversalRxs.tolist()
# # print("list of reactions involved in neuroactive production/consumption in Universal Model: \n", NeuroUniversalRxs)
# print("number of reactions:", len(NeuroUniversalRxs))

,0
0,34DHPHEtex
1,34DHPHEtpp
2,3A2OA
3,3HLYTCL
4,3MBAt6
...,...
360,VALTA
361,araphe3
362,aratry1
363,aratry2


In [5]:
gramModel=read_sbml_model("universe_gramneg.xml")

ModelMetabs=[]
ReactionsIdData=[]
ReactionsIdDataIndexNames=[]
TransportersINneuroRXs=[]
TransptGPRList=[]
UMetabList=[]

for x in gramModel.metabolites: #for each metabolite of the model metabolites
    UMetabList.append(x.id)

for NeuroactiveMetab in NeuroMetab:
    NeuroactiveMetab=NeuroactiveMetab.replace(" ' ", '')
    
    reactionsID=[]
    genesList=[]

    producingDF=gramModel.metabolites.get_by_id(NeuroactiveMetab).summary().producing_flux
    consumingDF=gramModel.metabolites.get_by_id(NeuroactiveMetab).summary().consuming_flux

    if producingDF.empty==False:
        slicedProducingDF=producingDF[['reaction']]
        listOfProdStrings = ['producing'] * slicedProducingDF.shape[0]
        slicedProducingDF.insert(1, "RxType", listOfProdStrings, True)

    if consumingDF.empty==False:
        slicedConsumingDF=consumingDF[['reaction']]
        listOfConsumStrings = ['consuming'] * slicedConsumingDF.shape[0]
        slicedConsumingDF.insert(1, "RxType", listOfConsumStrings, True)

        AllRXsDF = pd.concat([slicedProducingDF, slicedConsumingDF])
# print('dataframe with all the reactions \n:', AllRXsDF)

        for index, rows in AllRXsDF.iterrows():    # Put REACTIONids in a list so that we can later loop over it
            RowIndex = str(NeuroactiveMetab)
            Row =[rows.reaction, RowIndex, rows.RxType]
            ReactionsIdData.append(Row)

ReactionsIdDf=pd.DataFrame(ReactionsIdData, columns=['reaction',  'metabolite', 'RxType'])
ReactionsIdDf=ReactionsIdDf.sort_values('reaction')
print('dataframe with reactions \n:', ReactionsIdDf)

dataframe with reactions 
:        reaction metabolite     RxType
25   34DHPHEtex  34dhphe_p  producing
27   34DHPHEtex  34dhphe_e  producing
28   34DHPHEtex  34dhphe_e  consuming
30   34DHPHEtpp  34dhphe_c  producing
26   34DHPHEtpp  34dhphe_p  consuming
..          ...        ...        ...
130       VALTA   glu__L_c  producing
15      araphe3   trp__L_c  producing
16      aratry1   trp__L_c  producing
22      aratry2   trp__L_c  consuming
17      aratyr4   trp__L_c  producing

[447 rows x 3 columns]


In [11]:
gramRxs=ReactionsIdDf.reaction.unique()
gramRxs=gramRxs.tolist()
allRxs=NeuroUniversalRxs+gramRxs
allRxs = list(set(allRxs))
print("number of reactions:", len(allRxs))

allNeuroRxsIds = pd.DataFrame(allRxs)
allNeuroRxsIds=allNeuroRxsIds.rename(columns = {0:'ReactionID'})
# allNeuroRxsIds.to_excel("allNeuroRxsIds.xlsx")

number of reactions: 367
